In [114]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
df_stu, meta_stu = pyreadstat.read_sav('data/CY08MSP_STU_QQQ_HUN.sav')

In [116]:
import os
with open('data/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [142]:
df_stu['STUD_MATH'] = 0
cols = utils.meta2docs(meta_stu, excluded=['test','CNTRYID'])

cols.append( Document(
                    page_content= 'Student\'s score in Mathematics',
                    metadata={"year": 2022, "original_col_name": 'STUD_MATH'},
                ))

cols.append( Document(
                    page_content= 'Student\'s score in Reading',
                    metadata={"year": 2022, "original_col_name": 'STUD_READ'},
                ))

cols.append( Document(
                    page_content= 'Student\'s score in Science',
                    metadata={"year": 2022, "original_col_name": 'STUD_SCIE'},
                ))
#drop empty columns
df_stu = df_stu.dropna(axis=1, how='all')

cols2 = []
for col in cols:
    if col.metadata['original_col_name'] in df_stu.columns:
        cols2.append(col)
cols = cols2

cols_vectorstore = FAISS.from_documents(cols, OpenAIEmbeddings())

cols_retriever = cols_vectorstore.as_retriever()

/tmp/ipykernel_223909/789602455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stu['STUD_MATH'] = 0


1008

In [155]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache


llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))


In [125]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give  linear regression coefficients with standard errors as result.', df_stu, meta_stu, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['STUD_MATH', 'STUD_SCIE', 'STUD_READ', 'MATHMOT']
content="def run(data):\n    data = data[['ESCS', 'STUD_MATH']].dropna()\n    \n    X = data['ESCS'].values.reshape(-1,1)\n    y = data['STUD_MATH'].values\n    \n    model = LinearRegression()\n    model.fit(X, y)\n    \n    coef = model.coef_[0]\n    std_err = np.sqrt(np.mean((y - model.predict(X))**2)/(len(X) - 2)) / np.sqrt(np.var(X))\n    \n    return coef, std_err" response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 280, 'total_tokens': 389}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-bce0dc05-a434-4d4e-b114-b357c70fd885-0' usage_metadata={'input_tokens': 280, 'output_tokens': 109, 'total_tokens': 389}
def run(data):
    data = data[['ESCS', 'STUD_MATH']].dropna()
    
    X = data['ESCS'].values.reshape(-1,1)
    y = data['STUD_M

{'question': 'What is the relation between social background and math score of students? Please give  linear regression coefficients with standard errors as result.',
 'result': array([47.26859646,  1.11229368]),
 'hint_cols': ['ESCS',
  'SOCONPA',
  'COBN_M',
  'COBN_F',
  'STUD_MATH',
  'STUD_SCIE',
  'STUD_READ',
  'MATHMOT']}

In [126]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please answer with some exact values.', df_stu, meta_stu, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['STUD_MATH', 'STUD_SCIE', 'STUD_READ', 'MATHMOT']
content="def run(data):\n    data = data[['ESCS', 'STUD_MATH']].dropna()\n    \n    correlation = data['ESCS'].corr(data['STUD_MATH'])\n    \n    return correlation" response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 275, 'total_tokens': 315}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-11f76185-8a26-4602-ad95-5b5fde51b274-0' usage_metadata={'input_tokens': 275, 'output_tokens': 40, 'total_tokens': 315}
def run(data):
    data = data[['ESCS', 'STUD_MATH']].dropna()
    
    correlation = data['ESCS'].corr(data['STUD_MATH'])
    
    return correlation
['ESCS', 'STUD_MATH']
Iterating "STUD_MATH" 
[0.4788089210791027, 0.4758693053729336, 0.4791414020402215, 0.48582388249150876, 0.4840002529809444]


{'question': 'What is the relation between social background and math score of students? Please answer with some exact values.',
 'result': 0.48072875279294214,
 'hint_cols': ['ESCS',
  'SOCONPA',
  'COBN_M',
  'COBN_F',
  'STUD_MATH',
  'STUD_SCIE',
  'STUD_READ',
  'MATHMOT']}

In [131]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please answer with some exact values.', df_stu, meta_stu, cols_retriever)

Social background, Math score
hint Social background
['IC177Q02JA', 'FL174Q02JA', 'ESCS', 'IC178Q02JA']
hint  Math score
['STUD_MATH', 'STUD_SCIE', 'STUD_READ', 'MATHMOT']
content="def run(data):\n    result, statistical_ = data[['ESCS', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson correlation'\n    return result, statistical_" response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 344, 'total_tokens': 387}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-9fe1bf91-bbbf-4b75-8940-4be7d7e0bda5-0' usage_metadata={'input_tokens': 344, 'output_tokens': 43, 'total_tokens': 387}
def run(data):
    result, statistical_ = data[['ESCS', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson correlation'
    return result, statistical_
['ESCS', 'STUD_MATH']
Iterating "STUD_MATH" 
[(0.47880892107910455, 'Pearson correlation'), (0.4758693053729348, 'Pearson correlation'), (0.47914140204022543, 'Pearson correlation

{'question': 'What is the relation between social background and math score of students? Please answer with some exact values.',
 'result': (0.4807287527929439, 'Pearson correlation'),
 'used_columns': ['ESCS', 'STUD_MATH'],
 'hint_cols': ['IC177Q02JA',
  'FL174Q02JA',
  'ESCS',
  'IC178Q02JA',
  'STUD_MATH',
  'STUD_SCIE',
  'STUD_READ',
  'MATHMOT']}

In [132]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please answer with some exact values, that show the direction and strenght of the relation.', df_stu, meta_stu, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['STUD_MATH', 'STUD_SCIE', 'STUD_READ', 'MATHMOT']
content="def run(data):\n    data = data[['ESCS', 'STUD_MATH']].dropna()\n    \n    correlation = data['ESCS'].corr(data['STUD_MATH'])\n    \n    return correlation, 'Pearson correlation'" response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 309, 'total_tokens': 355}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d02c6cd0-463d-4061-b73b-9caa620c4ce7-0' usage_metadata={'input_tokens': 309, 'output_tokens': 46, 'total_tokens': 355}
def run(data):
    data = data[['ESCS', 'STUD_MATH']].dropna()
    
    correlation = data['ESCS'].corr(data['STUD_MATH'])
    
    return correlation, 'Pearson correlation'
['ESCS', 'STUD_MATH']
Iterating "STUD_MATH" 
[(0.4788089210791027, 'Pearson correlation'), (0.4758693053729336, 'Pearson correlation'), (0.4791

{'question': 'What is the relation between social background and math score of students? Please answer with some exact values, that show the direction and strenght of the relation.',
 'result': (0.48072875279294214, 'Pearson correlation'),
 'used_columns': ['ESCS', 'STUD_MATH'],
 'hint_cols': ['ESCS',
  'SOCONPA',
  'COBN_M',
  'COBN_F',
  'STUD_MATH',
  'STUD_SCIE',
  'STUD_READ',
  'MATHMOT']}

In [133]:
utils.pipeline(llm, 'Is social background positive or negative effect on math score of students? Please answer with some exact values.', df_stu, meta_stu, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['STUD_MATH', 'STUD_SCIE', 'STUD_READ', 'MATHPREF']
content="def run(data):\n    result, statistical_ = data[['ESCS', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson correlation'\n    return result, statistical_" response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 297, 'total_tokens': 340}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-27f1c84c-9964-406a-82e8-21a59d95f540-0' usage_metadata={'input_tokens': 297, 'output_tokens': 43, 'total_tokens': 340}
def run(data):
    result, statistical_ = data[['ESCS', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson correlation'
    return result, statistical_
['ESCS', 'STUD_MATH']
Iterating "STUD_MATH" 
[(0.47880892107910455, 'Pearson correlation'), (0.4758693053729348, 'Pearson correlation'), (0.47914140204022543, 'Pearson correlation'), (0.485

{'question': 'Is social background positive or negative effect on math score of students? Please answer with some exact values.',
 'result': (0.4807287527929439, 'Pearson correlation'),
 'used_columns': ['ESCS', 'STUD_MATH'],
 'hint_cols': ['ESCS',
  'SOCONPA',
  'COBN_M',
  'COBN_F',
  'STUD_MATH',
  'STUD_SCIE',
  'STUD_READ',
  'MATHPREF']}

In [134]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please answer with some exact values, using regression.', df_stu, meta_stu, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['STUD_MATH', 'STUD_SCIE', 'STUD_READ', 'MATHMOT']
content="def run(data):\n    import pandas as pd\n    from sklearn.linear_model import LinearRegression\n    \n    data = data[['ESCS', 'STUD_MATH']].dropna()\n    \n    X = data[['ESCS']]\n    y = data['STUD_MATH']\n    \n    model = LinearRegression()\n    model.fit(X, y)\n    \n    return (model.coef_[0], 'Linear Regression')" response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 300, 'total_tokens': 381}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-32d51ab8-6cc8-4de8-9ed2-28de1444720c-0' usage_metadata={'input_tokens': 300, 'output_tokens': 81, 'total_tokens': 381}
def run(data):
    import pandas as pd
    from sklearn.linear_model import LinearRegression
    
    data = data[['ESCS', 'STUD_MATH']].dropna()
    
    X = data[['ESCS']]


{'question': 'What is the relation between social background and math score of students? Please answer with some exact values, using regression.',
 'result': (47.26859645731944, 'Linear Regression'),
 'used_columns': ['ESCS', 'STUD_MATH'],
 'hint_cols': ['ESCS',
  'SOCONPA',
  'COBN_M',
  'COBN_F',
  'STUD_MATH',
  'STUD_SCIE',
  'STUD_READ',
  'MATHMOT']}

In [135]:
utils.pipeline(llm, 'What is the relation between number of books at home and math score of students? Please answer with some exact values, that show the direction and strenght of the relation.', df_stu, meta_stu, cols_retriever)

Number of books at home, Math score of students
hint Number of books at home
['ST255Q01JA', 'PA195Q01JA', 'ST256Q10JA', 'ST256Q06JA']
hint  Math score of students
['STUD_MATH', 'STUD_SCIE', 'STUD_READ', 'AGE']
content="def run(data):\n    data = data[['ST255Q01JA', 'PA195Q01JA', 'STUD_MATH']].dropna()\n    \n    correlation = data['ST255Q01JA'].corr(data['STUD_MATH'])\n    \n    return correlation, 'Pearson Correlation'" response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 338, 'total_tokens': 398}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-8f17538e-e116-4992-a716-0722a52b6ad7-0' usage_metadata={'input_tokens': 338, 'output_tokens': 60, 'total_tokens': 398}
def run(data):
    data = data[['ST255Q01JA', 'PA195Q01JA', 'STUD_MATH']].dropna()
    
    correlation = data['ST255Q01JA'].corr(data['STUD_MATH'])
    
    return correlation, 'Pearson Correlation'
['ST255Q01JA', 'PA195Q01JA', 'STUD_MATH']


{'question': 'What is the relation between number of books at home and math score of students? Please answer with some exact values, that show the direction and strenght of the relation.',
 'result': (nan, 'Pearson Correlation'),
 'used_columns': ['ST255Q01JA', 'PA195Q01JA', 'STUD_MATH'],
 'hint_cols': ['ST255Q01JA',
  'PA195Q01JA',
  'ST256Q10JA',
  'ST256Q06JA',
  'STUD_MATH',
  'STUD_SCIE',
  'STUD_READ',
  'AGE']}

In [160]:
utils.pipeline(llm, 'What is the relation between number of books at home and math score of students? Please answer with some exact values, using regression.', df_stu, meta_stu, cols_retriever)

Number of books at home, Math score
hint Number of books at home
['ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA', 'ST256Q09JA']
hint  Math score
['STUD_MATH', 'MATHMOT', 'MATHPREF', 'ST296Q01JA']
content="def run(data):\n    data = data[['ST255Q01JA', 'STUD_MATH']].dropna()\n    \n    result, _ = stats.pearsonr(data['ST255Q01JA'], data['STUD_MATH'])\n    \n    return result, 'Pearson Correlation'" response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 371, 'total_tokens': 430}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7804ff3c-8341-4bcf-b427-bd55ad788003-0' usage_metadata={'input_tokens': 371, 'output_tokens': 59, 'total_tokens': 430}
def run(data):
    data = data[['ST255Q01JA', 'STUD_MATH']].dropna()
    
    result, _ = stats.pearsonr(data['ST255Q01JA'], data['STUD_MATH'])
    
    return result, 'Pearson Correlation'
['ST255Q01JA', 'STUD_MATH']
Iterating "STUD_MATH" 


NameError: name 'stats' is not defined

In [137]:
utils.pipeline(llm, 'What is the relation between number of books at home and math score of students? Please answer with some exact values, that show the direction and strenght of the relation.', df_stu, meta_stu, cols_retriever)

Number of books at home, Math score of students
hint Number of books at home
['PA195Q01JA', 'ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA']
hint  Math score of students
['STUD_MATH', 'STUD_SCIE', 'STUD_READ', 'AGE']
content="def run(data):\n    data = data[['PA195Q01JA', 'STUD_MATH']].dropna()\n    \n    correlation = data['PA195Q01JA'].corr(data['STUD_MATH'])\n    \n    return correlation, 'Pearson Correlation'" response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 338, 'total_tokens': 391}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a7a2c1be-d9f8-4628-a57c-e816886e384d-0' usage_metadata={'input_tokens': 338, 'output_tokens': 53, 'total_tokens': 391}
def run(data):
    data = data[['PA195Q01JA', 'STUD_MATH']].dropna()
    
    correlation = data['PA195Q01JA'].corr(data['STUD_MATH'])
    
    return correlation, 'Pearson Correlation'
['PA195Q01JA', 'STUD_MATH']
Iterating "STUD_MATH" 
[(nan, 'Pearson Cor

{'question': 'What is the relation between number of books at home and math score of students? Please answer with some exact values, that show the direction and strenght of the relation.',
 'result': (nan, 'Pearson Correlation'),
 'used_columns': ['PA195Q01JA', 'STUD_MATH'],
 'hint_cols': ['PA195Q01JA',
  'ST255Q01JA',
  'ST256Q10JA',
  'ST256Q06JA',
  'STUD_MATH',
  'STUD_SCIE',
  'STUD_READ',
  'AGE']}

In [144]:
utils.pipeline(llm, 'What is the relation between number of books at home and math score of students? Please answer with some exact values, using regression.', df_stu, meta_stu, cols_retriever)


Number of books at home, Math score
hint Number of books at home
['ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA', 'ST256Q09JA']
hint  Math score
['STUD_MATH', 'MATHMOT', 'MATHPREF', 'ST296Q01JA']
content="def run(data):\n    data = data[['ST255Q01JA', 'STUD_MATH']].dropna()\n    \n    X = data['ST255Q01JA'].values.reshape(-1,1)\n    y = data['STUD_MATH'].values\n    \n    from sklearn.linear_model import LinearRegression\n    model = LinearRegression()\n    model.fit(X, y)\n    \n    return (model.coef_[0], 'Linear Regression')" response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 371, 'total_tokens': 460}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-921a173e-5941-470e-86d4-251438d9bb20-0' usage_metadata={'input_tokens': 371, 'output_tokens': 89, 'total_tokens': 460}
def run(data):
    data = data[['ST255Q01JA', 'STUD_MATH']].dropna()
    
    X = data['ST255Q01JA'].values.reshape(-1,1)
    y = data['ST

{'question': 'What is the relation between number of books at home and math score of students? Please answer with some exact values, using regression.',
 'result': (26.99464822449164, 'Linear Regression'),
 'used_columns': ['ST255Q01JA', 'STUD_MATH'],
 'hint_cols': ['ST255Q01JA',
  'ST256Q10JA',
  'ST256Q06JA',
  'ST256Q09JA',
  'STUD_MATH',
  'MATHMOT',
  'MATHPREF',
  'ST296Q01JA']}

In [148]:
utils.pipeline(llm, 'What is the relation between number of books at home and math score of students? Please answer with some exact values.', df_stu, meta_stu, cols_retriever)


Number of books at home, Math score of students
hint Number of books at home
['ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA', 'ST256Q09JA']
hint  Math score of students
['STUD_MATH', 'AGE', 'ST001D01T', 'GRADE']
content="def run(data):\n    result, statistical_ = data[['ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA', 'ST256Q09JA', 'STUD_MATH']].dropna().corr()['ST255Q01JA']['STUD_MATH'], 'Pearson Correlation'\n    return result, statistical_" response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 348, 'total_tokens': 419}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-587308e3-c490-4083-a44c-79cdad76a180-0' usage_metadata={'input_tokens': 348, 'output_tokens': 71, 'total_tokens': 419}
def run(data):
    result, statistical_ = data[['ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA', 'ST256Q09JA', 'STUD_MATH']].dropna().corr()['ST255Q01JA']['STUD_MATH'], 'Pearson Correlation'
    return result, statistical_
['ST255Q01JA', 'S

{'question': 'What is the relation between number of books at home and math score of students? Please answer with some exact values.',
 'result': (0.419726653773834, 'Pearson Correlation'),
 'inter': 'The result of 0.419726653773834 indicates a moderate positive correlation between the number of books at home and math scores of students. This means that as the number of books at home increases, the math scores of students also tend to increase. The Pearson correlation coefficient ranges from -1 to 1, with 1 indicating a perfect positive correlation, 0 indicating no correlation, and -1 indicating a perfect negative correlation. In this case, a value of 0.42 suggests a moderate positive relationship between the two variables.',
 'used_columns': ['ST255Q01JA',
  'ST256Q10JA',
  'ST256Q06JA',
  'ST256Q09JA',
  'STUD_MATH'],
 'hint_cols': ['ST255Q01JA',
  'ST256Q10JA',
  'ST256Q06JA',
  'ST256Q09JA',
  'STUD_MATH',
  'AGE',
  'ST001D01T',
  'GRADE']}

In [149]:
utils.pipeline(llm, 'What is the relation between family salary income and math score of students? Please answer with some exact values.', df_stu, meta_stu, cols_retriever)

Family salary income, Math score of students
hint Family salary income
['ST295Q03JA', 'WB164Q01HA', 'ST295Q04JA', 'ST294Q03JA']
hint  Math score of students
['STUD_MATH', 'AGE', 'ST001D01T', 'GRADE']
content="def run(data):\n    result, statistical_ = data[['WB164Q01HA', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson correlation'\n    return result, statistical_" response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 357, 'total_tokens': 403}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-eda5305e-639f-44ad-a93b-54d71020b584-0' usage_metadata={'input_tokens': 357, 'output_tokens': 46, 'total_tokens': 403}
def run(data):
    result, statistical_ = data[['WB164Q01HA', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson correlation'
    return result, statistical_
['WB164Q01HA', 'STUD_MATH']
Iterating "STUD_MATH" 
[(-0.11132364420618834, 'Pearson correlation'), (-0.10902572058398009, 'Pearson correlat

{'question': 'What is the relation between family salary income and math score of students? Please answer with some exact values.',
 'result': (-0.11418679473236797, 'Pearson correlation'),
 'inter': 'The result of -0.11418679473236797 indicates a weak negative correlation between family salary income and math scores of students. This means that as family salary income increases, math scores tend to decrease slightly. The Pearson correlation coefficient ranges from -1 to 1, with -1 indicating a perfect negative correlation, 0 indicating no correlation, and 1 indicating a perfect positive correlation. In this case, the correlation is close to 0, suggesting a very weak relationship between family salary income and math scores.',
 'used_columns': ['WB164Q01HA', 'STUD_MATH'],
 'hint_cols': ['ST295Q03JA',
  'WB164Q01HA',
  'ST295Q04JA',
  'ST294Q03JA',
  'STUD_MATH',
  'AGE',
  'ST001D01T',
  'GRADE']}

In [150]:
utils.pipeline(llm, 'What is the relation between family financial situation and math score of students? Please answer with some exact values.', df_stu, meta_stu, cols_retriever)

Family income, Family expenses, Family savings, Math score of students
hint Family income
['FAMSUP', 'WB164Q01HA', 'ST259Q01JA', 'WORKHOME']
hint  Family expenses
['WB164Q01HA', 'FAMSUP', 'ST261Q05JA', 'ST295Q03JA']
hint  Family savings
['FAMSUP', 'WB164Q01HA', 'HOMEPOS', 'WORKHOME']
hint  Math score of students
['STUD_MATH', 'AGE', 'ST001D01T', 'GRADE']
content="def run(data):\n    result, statistical_ = data[['WB164Q01HA', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson Correlation'\n    return result, statistical_" response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 601, 'total_tokens': 648}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f748a43c-19fc-4848-a586-8876510b7713-0' usage_metadata={'input_tokens': 601, 'output_tokens': 47, 'total_tokens': 648}
def run(data):
    result, statistical_ = data[['WB164Q01HA', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson Correlation'
    return res

{'question': 'What is the relation between family financial situation and math score of students? Please answer with some exact values.',
 'result': (1.0, 'Pearson Correlation'),
 'inter': "The result of (1.0, 'Pearson Correlation') indicates a perfect positive correlation between family financial situation and math scores of students. This means that as family financial situation increases, math scores of students also increase in a linear fashion. A Pearson correlation coefficient of 1.0 signifies a strong relationship between the two variables, suggesting that students from wealthier families tend to have higher math scores.",
 'used_columns': ['WB164Q01HA', 'WB164Q01HA', 'WB164Q01HA', 'STUD_MATH'],
 'hint_cols': ['FAMSUP',
  'WB164Q01HA',
  'ST259Q01JA',
  'WORKHOME',
  'WB164Q01HA',
  'FAMSUP',
  'ST261Q05JA',
  'ST295Q03JA',
  'FAMSUP',
  'WB164Q01HA',
  'HOMEPOS',
  'WORKHOME',
  'STUD_MATH',
  'AGE',
  'ST001D01T',
  'GRADE']}

In [151]:
utils.pipeline(llm, 'What is the relation between family social status and math score of students? Please answer with some exact values.', df_stu, meta_stu, cols_retriever)

family_social_status, math_score
hint family_social_status
['ESCS', 'SOCONPA', 'BMMJ1', 'BFMJ2']
hint  math_score
['STUD_MATH', 'ST291Q08JA', 'ST296Q02JA', 'ST296Q03JA']
content="def run(data):\n    result, statistical_ = data[['SOCONPA', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson correlation'\n    return result, statistical_" response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 367, 'total_tokens': 411}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d9619bf3-353f-4bf4-94fb-f3e046cf153f-0' usage_metadata={'input_tokens': 367, 'output_tokens': 44, 'total_tokens': 411}
def run(data):
    result, statistical_ = data[['SOCONPA', 'STUD_MATH']].dropna().corr().iloc[0, 1], 'Pearson correlation'
    return result, statistical_
['SOCONPA', 'STUD_MATH']
Iterating "STUD_MATH" 
[(0.16797877041625428, 'Pearson correlation'), (0.16763086554162868, 'Pearson correlation'), (0.1695744841012936, 'Pearson corr

{'question': 'What is the relation between family social status and math score of students? Please answer with some exact values.',
 'result': (0.16757871229947513, 'Pearson correlation'),
 'inter': "The result provided is a Pearson correlation coefficient of 0.16757871229947513. This value indicates a weak positive correlation between family social status and math scores of students. In other words, as family social status increases, there is a slight tendency for math scores to also increase. However, the correlation is not very strong, suggesting that other factors may also play a significant role in determining students' math scores.",
 'used_columns': ['SOCONPA', 'STUD_MATH'],
 'hint_cols': ['ESCS',
  'SOCONPA',
  'BMMJ1',
  'BFMJ2',
  'STUD_MATH',
  'ST291Q08JA',
  'ST296Q02JA',
  'ST296Q03JA']}

In [152]:
utils.pipeline(llm, 'What is the relation between family social background and math score of students? Please answer with some exact values.', df_stu, meta_stu, cols_retriever)

family_social_background, math_score
hint family_social_background
['SOCONPA', 'FAMSUP', 'COBN_F', 'COBN_M']
hint  math_score
['STUD_MATH', 'ST291Q08JA', 'ST296Q02JA', 'ST296Q03JA']
content="def run(data):\n    result, statistical_method = data[['SOCONPA', 'FAMSUP', 'STUD_MATH']].dropna().corr().iloc[0, 2], 'Pearson correlation'\n    return result, statistical_method" response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 341, 'total_tokens': 390}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1eef5154-78f9-4679-a13b-4633e006abc9-0' usage_metadata={'input_tokens': 341, 'output_tokens': 49, 'total_tokens': 390}
def run(data):
    result, statistical_method = data[['SOCONPA', 'FAMSUP', 'STUD_MATH']].dropna().corr().iloc[0, 2], 'Pearson correlation'
    return result, statistical_method
['SOCONPA', 'FAMSUP', 'STUD_MATH']
Iterating "STUD_MATH" 
[(0.16465439579312352, 'Pearson correlation'), (0.16398301326

{'question': 'What is the relation between family social background and math score of students? Please answer with some exact values.',
 'result': (0.16325038223351035, 'Pearson correlation'),
 'inter': "The result provided is a Pearson correlation coefficient of 0.163, which indicates a weak positive correlation between family social background and math scores of students. This means that as family social background increases, math scores also tend to increase slightly. However, the correlation is not very strong, suggesting that other factors may also play a significant role in determining students' math scores.",
 'used_columns': ['SOCONPA', 'FAMSUP', 'STUD_MATH'],
 'hint_cols': ['SOCONPA',
  'FAMSUP',
  'COBN_F',
  'COBN_M',
  'STUD_MATH',
  'ST291Q08JA',
  'ST296Q02JA',
  'ST296Q03JA']}

In [157]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give  linear regression coefficients with standard errors as result.', df_stu, meta_stu, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['STUD_MATH', 'MATHMOT', 'MATHPREF', 'ST296Q01JA']
content="def run(data):\n    data = data[['ESCS', 'SOCONPA', 'STUD_MATH']].dropna()\n    \n    X = data[['ESCS', 'SOCONPA']]\n    y = data['STUD_MATH']\n    \n    model = sm.OLS(y, sm.add_constant(X)).fit()\n    \n    return model.params, 'Linear Regression'" response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 331, 'total_tokens': 405}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-25133cac-3208-424e-bff2-6e525cc59296-0' usage_metadata={'input_tokens': 331, 'output_tokens': 74, 'total_tokens': 405}
def run(data):
    data = data[['ESCS', 'SOCONPA', 'STUD_MATH']].dropna()
    
    X = data[['ESCS', 'SOCONPA']]
    y = data['STUD_MATH']
    
    model = sm.OLS(y, sm.add_constant(X)).fit()
    
    return model.params, 'Linear Regression'
['ES

{'question': 'What is the relation between social background and math score of students? Please give  linear regression coefficients with standard errors as result.',
 'result': (const      478.016144
  ESCS        44.945197
  SOCONPA      8.063908
  dtype: float64,
  'Linear Regression'),
 'inter': 'The linear regression coefficients with standard errors for the relation between social background and math score of students are as follows:\n\n- Constant: 478.016144\n- ESCS (Index of Economic, Social, and Cultural Status): 44.945197\n- SOCONPA (Social Contact and Participation): 8.063908\n\nThese coefficients indicate that there is a positive relationship between social background (ESCS and SOCONPA) and math scores of students. Specifically, for every one unit increase in ESCS, the math score is expected to increase by approximately 44.95 points, and for every one unit increase in SOCONPA, the math score is expected to increase by approximately 8.06 points.',
 'used_columns': ['ESCS', '

In [161]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give  linear regression coefficients with standard errors as result.', df_stu, meta_stu, cols_retriever)

Social background, Math score
hint Social background
['ESCS', 'SOCONPA', 'COBN_M', 'COBN_F']
hint  Math score
['STUD_MATH', 'MATHMOT', 'MATHPREF', 'ST296Q01JA']
content="def run(data):\n    data = data[['ESCS', 'SOCONPA', 'STUD_MATH']].dropna()\n    \n    X = data[['ESCS', 'SOCONPA']]\n    y = data['STUD_MATH']\n    \n    model = sm.OLS(y, sm.add_constant(X)).fit()\n    \n    return model.params, 'Linear Regression'" response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 331, 'total_tokens': 405}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-25133cac-3208-424e-bff2-6e525cc59296-0' usage_metadata={'input_tokens': 331, 'output_tokens': 74, 'total_tokens': 405}
def run(data):
    data = data[['ESCS', 'SOCONPA', 'STUD_MATH']].dropna()
    
    X = data[['ESCS', 'SOCONPA']]
    y = data['STUD_MATH']
    
    model = sm.OLS(y, sm.add_constant(X)).fit()
    
    return model.params, 'Linear Regression'
['ES

{'question': 'What is the relation between social background and math score of students? Please give  linear regression coefficients with standard errors as result.',
 'result': (const      478.016144
  ESCS        44.945197
  SOCONPA      8.063908
  dtype: float64,
  'Linear Regression'),
 'inter': 'The linear regression coefficients with standard errors for the relation between social background and math score of students are as follows:\n\n- Constant: 478.016144\n- ESCS (Index of Economic, Social, and Cultural Status): 44.945197\n- SOCONPA (Social Contact and Participation): 8.063908\n\nThese coefficients indicate that there is a positive relationship between social background (ESCS and SOCONPA) and math scores of students. Specifically, for every one unit increase in ESCS, the math score is expected to increase by approximately 44.95 points, and for every one unit increase in SOCONPA, the math score is expected to increase by approximately 8.06 points.',
 'used_columns': ['ESCS', '